In [6]:
import random
from sympy import isprime

import random

N = 32

def miller_rabin(n, k): # number of tests to run
    if n == 2 or n == 3:
        return True

    if n <= 1 or n % 2 == 0:
        return False

    # write n-1 as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # witness loop
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def generate_large_prime(n_bits, k=10):
    while True:
        p = random.getrandbits(n_bits)
        if miller_rabin(p, k):
            return p

# Generate a large prime of approximately 2^32 size
is_prime1 = False
is_prime2 = False

while not is_prime1 and not is_prime2:

    # Generate two large primes
    prime1 = generate_large_prime(N//2)
    prime2 = generate_large_prime(N//2)

    # Test if:
    # 1. They are not equal
    # 2. They are both prime
    # 3. Their product is less than 2^64
    if (
        prime1 != prime2 and 
        isprime(prime1) and 
        isprime(prime2) and 
        prime1 * prime2 < 2 ** N
    ):
        
        is_prime1 = True
        is_prime2 = True

semiprime = prime1 * prime2

f"Factorization Target {prime1} \u00d7 {prime2} = {semiprime}"

'Factorization Target 25031 × 57839 = 1447768009'

In [7]:
from jung_kim_pmultiplier import JK_Multiplier
from tqdm import tqdm

pm = JK_Multiplier(n=N,N=semiprime)

pbar = tqdm(desc=f"Cracking {N}-bit Semiprime", unit="iter", dynamic_ncols=True)

while True:

    pbar.update()

    if pm.loop():

        if semiprime % pm._bin_to_int(pm.X) == 0:
            print(f"Factorization found {pm._bin_to_int(pm.X)} \u00d7 {semiprime // pm._bin_to_int(pm.X)} = {semiprime}")
        elif semiprime % pm._bin_to_int(pm.Y) == 0:
            print(f"Factorization found {pm._bin_to_int(pm.Y)} \u00d7 {semiprime // pm._bin_to_int(pm.Y)} = {semiprime}")
        else:
            print("Error")

        break

Cracking 40-bit Semiprime: 332295iter [11:51, 466.97iter/s] 


Factorization found 57839 × 25031 = 1447768009
